In [1]:
import math
import numpy as np

In [9]:
def max_not_diagonal(matrix):
    a = np.array(matrix)
    i,j = np.indices(a.shape)
    a[i >= j] = 0
    index = np.unravel_index(np.abs(a).argmax(), a.shape)
    return index, a[index]

def build_rotate_matrix(a_k, index, eps=1e-6):
    i, j = index
    U = np.zeros(a_k.shape)
    
    if abs(a_k[i, i] - a_k[j, j]) < eps:
        phi = math.phi / 4.0
        
    else:
        phi = 0.5 * math.atan(2.0 * a_k[i, j] / (a_k[i, i] - a_k[j, j]))
    U[i, j] = - math.sin(phi)
    U[j, i] = math.sin(phi)
    U[i, i] = math.cos(phi)
    U[j, j] = math.cos(phi)
    for m in range(len(U)):
        if m != i and m != j:
            U[m, m] = 1
    return U
    
def calc_stop(matrix):
    i,j = np.indices(matrix.shape)
    return (matrix[i < j] ** 2).sum() ** 0.5

def sum_not_diagonal(matrix):
    i,j = np.indices(matrix.shape)
    return 2 * (matrix[i < j] ** 2).sum()
    
def sum_diagonal(matrix):
    return (matrix.diagonal() ** 2).sum()
    
def find_accuracy(inp_matrix, eig_vals, eig_vectors):
    for eig_val, eig_vec in zip(eig_vals, eig_vectors):
        print("For eig_val = {} and eig_vec = {}".format(eig_val, eig_vec))
        x = inp_matrix.dot(eig_vec) - eig_val * eig_vec
        print("Error vector = {} norm = {}".format(x, np.linalg.norm(x)))
        print("=" * 10)
        
    
def Jacobi(input_matrix, eps=1e-6, max_iteration=120, printed=True):
    for number_iteration in range(max_iteration):
        
        if number_iteration == 0:    
            Ak = np.array(input_matrix)
            
        index, element = max_not_diagonal(Ak)
        
        Uk = build_rotate_matrix(Ak, index)        
        if number_iteration == 0:
            U_lin_operator = np.array(Uk)
        else:
            U_lin_operator = U_lin_operator.dot(Uk)
        Ak = np.dot(Uk.T, Ak).dot(Uk)
        
        criterion = calc_stop(Ak)
        
        if printed:
            i, j = index
            print("number of iteration = {}".format(number_iteration))
            print("max element = {} index = {}".format(element, index))
            print("a = ",Ak[i,i], "b = ", Ak[j,j], "g = ", Ak[index])
            print("ang coef = {", Uk[i,i]," ", Uk[j,i]," ", Uk[i,j], " ", Uk[j,j],"}")
            print("diagonalized matrix = ")
            print(Ak)
            print("Sum all = {}".format(sum_not_diagonal(Ak) + sum_diagonal(Ak)))
            print("criterion = {}".format(criterion))
            print("=" * 10)

        if criterion < eps:
            return Ak.diagonal(), U_lin_operator
    else:
        raise ValueError("Diverges")
        

In [10]:
input_matrix = np.array([[6.81, 1.04, 1.03, 1.165], 
                   [1.04, 3.61, 1.3, 0.16],
                   [1.03, 1.3, 5.99, 2.1],
                   [1.165, 0.16, 2.1, 5.55]])

eig_val, eig_vec = Jacobi(input_matrix)

for val, vec in zip(eig_val, eig_vec.T):
    print("Eig val = {} -> vec = {}".format(val, vec))

number of iteration = 0
max element = 2.1 index = (2, 3)
a =  7.88149236324 b =  3.65850763676 g =  1.01047642476e-16
ang coef = { 0.74303153029   0.669256411994   -0.669256411994   0.74303153029 }
diagonalized matrix = 
[[  6.81000000e+00   1.04000000e+00   1.54500620e+00   1.76297628e-01]
 [  1.04000000e+00   3.61000000e+00   1.07302202e+00  -7.51148291e-01]
 [  1.54500620e+00   1.07302202e+00   7.88149236e+00   1.01047642e-16]
 [  1.76297628e-01  -7.51148291e-01   2.83410448e-16   3.65850764e+00]]
Sum all = 145.34145
criterion = 2.28370860663089
number of iteration = 1
max element = 1.5450061961715071 index = (0, 2)
a =  5.7104883679 b =  8.98100399534 g =  -6.68952740424e-16
ang coef = { 0.814745925052   -0.579818141844   0.579818141844   0.814745925052 }
diagonalized matrix = 
[[  5.71048837e+00   2.25178131e-01  -6.68952740e-16   1.43637774e-01]
 [  2.25178131e-01   3.61000000e+00   1.47725118e+00  -7.51148291e-01]
 [ -8.23993651e-17   1.47725118e+00   8.98100400e+00   1.02220563

In [ ]:
find_accuracy(input_matrix, eig_val, eig_vec.T)

In [12]:
input_matrix = np.array([[6.81, 1.04, 1.03, 1.165], 
                   [1.04, 3.61, 1.3, 0.16],
                   [1.03, 1.3, 5.99, 2.1],
                   [1.165, 0.16, 2.1, 5.55]])

def pow_method(matrix, eps=1e-6):
    index = 1
    
    lam1 = 1
    yk1 = np.ones(len(matrix))
    yk2 = matrix.dot(yk1)
    lam2 = yk2[index] / yk1[index]
    
    while abs(lam2 - lam1) > eps:
        lam1 = lam2
        yk1 = yk2
        yk2 = matrix.dot(yk1)
        lam2 = yk2[index] / yk1[index]
    yk2 = yk2 / np.linalg.norm(yk2)
    return lam2, yk2
    
    
eig_val_max, eig_vec_max = pow_method(input_matrix)
print("Max eig val = ", eig_val_max, "Eig vec = ", eig_vec_max)


Max eig val =  9.36267870365 Eig vec =  [ 0.57322744  0.25109256  0.5897066   0.51049874]
